In [1]:
from datetime import *
import urllib.request as req
import requests
import bs4
import time
import pandas as pd
import numpy as np

In [2]:
### 變數及參數設定 ###

# 網址
url_1 = "https://udn.com/search/word/2/%E8%A9%90%E4%BF%9D" # 詐保
url_2 = "https://udn.com/search/word/2/%E8%B2%AA%E6%B1%99" # 貪汙
url_3 = "https://udn.com/search/word/2/%E5%85%AC%E5%8B%99%E5%93%A1%20%E8%B2%AA%E6%B1%99" # 公務員 貪汙
url_4 = "https://udn.com/search/word/2/%E5%85%A7%E7%B7%9A%E4%BA%A4%E6%98%93" # 內線交易
url_5 = "https://udn.com/search/word/2/%E6%AC%A0%E7%A8%85%E5%A4%A7%E6%88%B6"  # 欠稅大戶

# 幾日前
days = 7

# 獲得現在日期時刻
today = datetime.today()
print(today)

# 某日日期
## today = datetime.strptime("2024-05-31 23:59:59", "%Y-%m-%d %H:%M:%S")

2025-05-15 15:11:01.427191


In [3]:
def UDN(url, days, today):
    ## 建立 list ##
    Key = []        # 關鍵字
    Title = []      # 標題
    Time = []       # 發表日期
    Text = []       # 內文
    URL = []        # 網址

    response = requests.get(url)
    original_page = bs4.BeautifulSoup(response.text, "html.parser")
    news_num = int(original_page.find("div", class_ = "search-total").span.string)
    pages = news_num//20+1
    
    for page in range(1, pages+1):
        main_page = requests.get(url + "/" + str(page)).text
    
        main_page = bs4.BeautifulSoup(main_page, "html.parser")
        key = main_page.find("div", class_ = "search-actions")
        key_name = key.find("div", class_ = "input-holder").input["value"] 
        # 也可以使用 key_name = key.find("input", {"type": "search"})["value"]

        label = main_page.find_all("div", class_ = "story-list__text")
        for i in range(20):
            title = label[i].find("h2").a.string
            time = label[i].find("div", class_ = "story-list__info").time.string
            time = datetime.strptime(time, "%Y-%m-%d %H:%M")
            news_url = label[i].find("h2").a["href"]

            sec = (today - time).total_seconds()
            days_sec = timedelta(days = days).total_seconds()

            if sec <= days_sec and sec >= 0:
                    Key.append(key_name), Title.append(title), Time.append(time), URL.append(news_url)

                    news_response = requests.get(news_url)
                    news_page = bs4.BeautifulSoup(news_response.text, "html.parser")
                    texts = news_page.find_all("p")
                    news = ""
                    for text in texts:
                        news += text.text
                    Text.append(news.replace("\n", "").replace("\t", "").replace("\r", ""))
            
            elif sec < 0:
                continue
                
            else:
                break
                
        else:
            continue
            
        break
            
    # 輸出 DataFrame
    data = pd.DataFrame({
        "關鍵字":Key,
        "標題":Title,
        "時間":Time,
        "內文":Text,
        "網址":URL
    })
    
    return data

In [4]:
UDN(url_1, days, today)

,關鍵字,標題,時間,內文,網址
0,詐保,《「我是有錢人」迷思769》保戶對保險公司提訴訟 必要性醫療、需經過6個月治療引衝突,2025-05-15 09:16:00,一直以來，就有不少保戶及業務員，對於保險公司的「決定不理賠」，存有非常多的質疑與不滿，進而向...,https://udn.com/news/story/121591/8740825
1,詐保,香港中文大學辦吃藥領獎金？ 台中婦險被詐百萬警阻,2025-05-12 10:05:00,詐騙集團又有新話術，上月間以「香港中文大學中醫院研究交流活動」為由，誆騙台中西屯區陳姓婦人配...,https://udn.com/news/story/7320/8733259
2,詐保,「背殺女」自稱繁花似錦愛情攻勢 警攔詐保住單身農民血汗錢,2025-05-09 17:57:00,彰化縣65歲陳姓未婚男子瀏覽臉書，被暱稱「繁花似錦」的網友搭訕，進而互相line，網友自稱積...,https://udn.com/news/story/7320/8729362


In [28]:
UDN(url_2, days, today)

,關鍵字,標題,時間,內文,網址
0,貪汙,駐美代表俞大㵢否認800萬買奢華品 認搭公務車賞櫻,2024-06-26 07:21:00,中華民國駐美代表俞大㵢遭控濫用公務資源享受奢華駐外待遇，他25日親上火線回應，認了搭公務車和...,https://udn.com/news/story/6656/8054899
1,貪汙,教造假情資合詐千萬 情報員判15年,2024-06-26 03:42:00,國防部軍事情報局上校蘇永玲（左）及中校蘇薪名（右黃衣者）。圖／本報資料照片軍情局上校江芳瑜、...,https://udn.com/news/story/7321/8054613
2,貪汙,涉貪汙罪 新埤2任鄉長被訴,2024-06-26 03:01:00,屏東縣新埤鄉長潘和源涉貪汙案，涉違反人員進用，檢方昨起訴並建請從重量刑，移審法院後，法院裁定...,https://udn.com/news/story/7321/8054668
3,貪汙,涉貪汙罪 新埤2任鄉長被訴,2024-06-25 23:25:00,屏東縣新埤鄉前鄉長林志成、現任鄉長潘和源任內涉違反人員進用規定，用不符機要薦任課員何男、潘男...,https://vip.udn.com/vip/story/121584/8054668
4,貪汙,教造假情資合詐千萬 情報員判15年,2024-06-25 23:13:00,軍情局上校江芳瑜、中校徐之心，涉透過國家一級情報單位「國防部電訊發展室」中校蘇薪名，營造蒐集...,https://vip.udn.com/vip/story/121523/8054613
5,貪汙,違反人員進用薪資挪移…新埤鄉長涉貪起訴 法院30萬交保,2024-06-25 19:26:00,屏東縣新埤鄉前任鄉長林志成、現任鄉長潘和源，兩人任內時違反人員進用，用不符機要薦任課員何男、...,https://udn.com/news/story/7321/8054173
6,貪汙,批謝國樑為罷免閃躲 NET：應為強盜案轉列偵字向市民道歉,2024-06-25 17:44:00,基隆東岸商場接管爭議主富公司（NET）控告市長謝國樑強盜等五罪，地檢署已從他字案轉為偵字案，...,https://udn.com/news/story/7321/8054005
7,貪汙,4機關聲請釋憲 藍委：民進黨要掩飾8年貪汙橫行,2024-06-25 17:20:00,針對監察院就國會改革法案向憲法法庭聲請釋憲及暫時處分，國民黨立委許宇甄今天表示，民進黨為了抵...,https://udn.com/news/story/124078/8053921
8,貪汙,前、現任新埤鄉長濫用人事、調換薪資涉貪汙 屏檢起訴建請從重量刑,2024-06-25 12:20:00,屏東縣新埤鄉前任鄉長林志成、現任鄉長潘和源，兩人任內時違反人員進用，兩人任內任用不符機要薦任...,https://udn.com/news/story/7321/8053104
9,貪汙,男酒駕遭攔查 偷塞2千元求警「不要抓我」下場更慘,2024-06-25 11:49:00,35歲越南籍黎姓男子去年8月間凌晨，在桃園市蘆竹區飲酒後騎車上路，半路遭警方攔查，結果員警聞...,https://udn.com/news/story/7321/8053003


In [34]:
UDN(url_3, days, today)

,關鍵字,標題,時間,內文,網址
0,公務員 貪汙,教造假情資合詐千萬 情報員判15年,2024-06-26 03:42:00,國防部軍事情報局上校蘇永玲（左）及中校蘇薪名（右黃衣者）。圖／本報資料照片軍情局上校江芳瑜、...,https://udn.com/news/story/7321/8054613
1,公務員 貪汙,教造假情資合詐千萬 情報員判15年,2024-06-25 23:13:00,軍情局上校江芳瑜、中校徐之心，涉透過國家一級情報單位「國防部電訊發展室」中校蘇薪名，營造蒐集...,https://vip.udn.com/vip/story/121523/8054613
2,公務員 貪汙,獨／文化部前科長索賄857萬 二審今駁上訴…貪官下場判3年8月,2024-06-25 10:34:00,蕭銘彬在任職文化部文資局水下科科長、事務科科長期間，利用申請向風電廠商施壓、推薦，洩漏投標訊...,https://udn.com/news/story/7321/8052729
3,公務員 貪汙,中壢警投資失利竟替幫派查個資賺佣金 懲戒法院判休職1年半,2024-06-19 12:00:00,桃園市警員李宥葳勾搭從事暴力討債的白姓幫派成員，約定如李「介紹客戶」，將給高於行情的報酬或佣...,https://udn.com/news/story/7320/8040572


In [35]:
UDN(url_4, days, today)

,關鍵字,標題,時間,內文,網址
0,內線交易,東洋藥品採購BNT爆內線交易 時任總座翻供稱三方會議無結果,2024-06-25 17:38:00,東洋藥品時任總經理施俊良等12人，遭控洽談德國BNT疫苗內線交易，不法所得計413萬元。士林...,https://udn.com/news/story/7321/8053995
1,內線交易,警戒！75%台灣民眾三個月內收到詐騙訊息 31%曾遭詐騙得逞,2024-06-25 17:25:00,近年詐騙集團猖獗且手法不斷推陳出新，為提升大眾的識詐知能，國泰世華銀行25日發表首份《反詐行...,https://udn.com/news/story/7239/8053949
2,內線交易,協助朱國榮逃亡 「超跑達人」出庭：後悔做了錯誤的決定,2024-06-24 15:14:00,牽涉多起重大金融弊案的「國寶集團」總裁朱國榮去年棄保潛逃，船長鍾明華、「超跑達人」周哲男涉協...,https://udn.com/news/story/7321/8051169
3,內線交易,涉內線交易 台通光電董事長兒李嘉豪延押2個月,2024-06-20 21:02:00,北市議員陳重文涉圖利台通光董事長李慶煌，檢廉追查出李慶煌兒子李嘉豪等人涉內線交易案，今年4月...,https://udn.com/news/story/7321/8044440
4,內線交易,陳重文貪汙「案外案」 李嘉豪內線獲利上億聲請延押,2024-06-19 10:42:00,檢廉偵辦台北市議員陳重文圖利台智光案，發現台智光母公司台通光電在搜索前公告要買回庫藏股，董事...,https://udn.com/news/story/7321/8040293


In [36]:
UDN(url_5, days, today)

,關鍵字,標題,時間,內文,網址
